In [1]:
from pyscf import gto, dft, scf, lib, cc, fci
import numpy as np
import scipy as sp

How to get integrals from other molecules!!!

https://sunqm.github.io/pyscf/_modules/pyscf/gto/moleintor.html

In [2]:
%matplotlib notebook

H_bond=0.74
R= 2.0 # distance of H2 --- R --- H2

Ha1=(0, 0, -H_bond)
Ha2= (0, 0, 0)

Hb1= (0,0,R)
Hb2= (0,0,R+H_bond)

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(*Ha1, marker='o', color='green', label='systemA')
ax.scatter(*Ha2, marker='o', color='green', label='systemA')
ax.scatter(*Hb1, marker='o', color='red', label='systemB')
ax.scatter(*Hb2, marker='o', color='red', label='systemB')
plt.legend()
plt.show()


<IPython.core.display.Javascript object>

In [3]:
mol_A = gto.Mole(atom='{} {} {} {}'.format('H',*Ha1) + "; " + '{} {} {} {}'.format('H',*Ha2),
               basis="sto3g",
               charge=0,
               spin=0)

print(mol_A.atom)
print('')
mol_A.build()

H 0 0 -0.74; H 0 0 0



In [4]:
mol_B = gto.Mole(atom='{} {} {} {}'.format('H',*Hb1) + "; " + '{} {} {} {}'.format('H',*Hb2),
               basis="sto3g",
               charge=0,
               spin=0)

print(mol_B.atom)
print('')
mol_B.build()

H 0 0 2.0; H 0 0 2.74



good paper (https://aip.scitation.org/doi/pdf/10.1063/1.1534090)

## ISOLATED system

$$v_{\mathrm{eff}}^{\mathrm{KS}}\left[\rho_{\mathrm{I}}\right](\boldsymbol{r})=v_{\mathrm{I}}^{\mathrm{nuc}}(\boldsymbol{r})+\int \frac{\rho_{\mathrm{I}}\left(\boldsymbol{r}^{\prime}\right)}{\left|\boldsymbol{r}-\boldsymbol{r}^{\prime}\right|} d \boldsymbol{r}^{\prime}+\left.\frac{\delta E_{\mathrm{xc}}[\rho]}{\delta \rho}\right|_{\rho=\rho_{\mathrm{I}}(\boldsymbol{r})}$$

## Embedded potential

$$v_{\mathrm{eff}}^{\mathrm{emb}}\left[\rho_{\mathrm{I}}, \rho_{\mathrm{II}}\right](\boldsymbol{r})=v_{\mathrm{II}}^{\mathrm{nuc}}(\boldsymbol{r})+\int \frac{\rho_{\mathrm{II}}\left(\boldsymbol{r}^{\prime}\right)}{\left|\boldsymbol{r}-\boldsymbol{r}^{\prime}\right|} d \boldsymbol{r}^{\prime}+\frac{\delta E_{\mathrm{xc}}^{\mathrm{nadd}}\left[\rho_{\mathrm{I}}, \rho_{\mathrm{II}}\right]}{\delta \rho_{\mathrm{I}}}+v_{T}\left[\rho_{\mathrm{I}}, \rho_{\mathrm{II}}\right](\boldsymbol{r})$$

## Coupled Equations:

$$\begin{array}{c}
{\left[-\frac{\nabla^{2}}{2}+v_{\mathrm{eff}}^{\mathrm{KS}}\left[\rho_{\mathrm{I}}\right](\boldsymbol{r})+v_{\mathrm{eff}}^{\mathrm{emb}}\left[\rho_{\mathrm{I}}, \rho_{\mathrm{II}}\right](\boldsymbol{r})\right] \phi_{i}^{(I)}(\boldsymbol{r})=\epsilon_{i}^{(I)} \phi_{i}^{(I)}(\boldsymbol{r})} \\
i=1, \ldots, N_{\mathrm{I}} / 2
\end{array}$$

AND

$$\begin{array}{c}
{\left[-\frac{\nabla^{2}}{2}+v_{\mathrm{eff}}^{\mathrm{KS}}\left[\rho_{\mathrm{II}}\right](\boldsymbol{r})+v_{\mathrm{eff}}^{\mathrm{emb}}\left[\rho_{\mathrm{II}}, \rho_{\mathrm{I}}\right](\boldsymbol{r})\right] \phi_{i}^{(I I)}(\boldsymbol{r})=\epsilon_{i}^{(I I)} \phi_{i}^{(I I)}(\boldsymbol{r})} \\
i=1, \ldots, N_{\mathrm{II}} / 2
\end{array}$$

# Overall
(https://quantum-dynamics-hub.github.io/workshop_2018/talks/Pavanello/Quantum-ESPRESSO_2018.pdf)

$$E_{\mathrm{FDE}}[\rho]=E\left[\rho_{I}\right]+E\left[\rho_{I I}\right]+T_{s}^{\mathrm{nadd}}\left[\rho_{I}, \rho_{I I}\right]+E_{x c}^{\mathrm{nadd}}\left[\rho_{I}, \rho_{I I}\right]+V_{\mathrm{Coul}}^{\mathrm{nadd}}\left[\rho_{I}, \rho_{I I}\right]$$

- The energy is nearly $E_{\text{total}}\approx E\left[\rho_{I}\right]+E\left[\rho_{I I}\right]$
- just need to add non additive part (aka the coupling!)

# Approximations of non-additive parts

$$\tilde{T}_{s}^{\text {nadd }}\left[\rho_{\mathrm{I}}, \rho_{\mathrm{II}}\right]=\tilde{T}_{s}\left[\rho_{\mathrm{I}}+\rho_{\mathrm{II}}\right]-\tilde{T}_{s}\left[\rho_{\mathrm{I}}\right]-\tilde{T}_{s}\left[\rho_{\mathrm{II}}\right]$$

and

$$\tilde{v}_{T}\left[\rho_{\mathrm{I}}, \rho_{\mathrm{II}}\right](\boldsymbol{r})=\left.\frac{\delta \tilde{T}_{s}[\rho]}{\delta \rho}\right|_{\rho=\rho_{\mathrm{tot}}(\boldsymbol{r})}-\left.\frac{\delta \tilde{T}_{s}[\rho]}{\delta \rho}\right|_{\rho=\rho_{\mathrm{I}}(\boldsymbol{r})}$$


where $\tilde{T}_{s}[\rho]$ refers to some approximate kinetic-energy functional.

[J. Chem. Theory Comput. 2018, 14, 1928−1942 Article]


$$\gamma^{\text{Total}}=\begin{bmatrix}
\gamma^{A} & 0\\
0 & \gamma^{B} 
\end{bmatrix}$$

$$F=\begin{bmatrix}
F^{AA} & F^{AB}\\
F^{BA} & F^{BB} 
\end{bmatrix}$$

$$S=\begin{bmatrix}
S^{AA} & S^{AB}\\
S^{BA} & S^{BB} 
\end{bmatrix}$$

$$\begin{array}{c}
{\left[-\frac{\nabla^{2}}{2}+v_{\mathrm{eff}}^{\mathrm{KS}}\left[\rho_{\mathrm{I}}\right](\boldsymbol{r})+v_{\mathrm{eff}}^{\mathrm{emb}}\left[\rho_{\mathrm{I}}, \rho_{\mathrm{II}}\right](\boldsymbol{r})\right] \phi_{i}^{(I)}(\boldsymbol{r})=\epsilon_{i}^{(I)} \phi_{i}^{(I)}(\boldsymbol{r})} \\
i=1, \ldots, N_{\mathrm{I}} / 2
\end{array}$$

from equation (6) of https://aip.scitation.org/doi/pdf/10.1063/1.1534090 [also 3 and 4!]

$$-\frac{\nabla^{2}}{2} + v_{\mathrm{I}}^{\mathrm{nuc}} + v_{\mathrm{II}}^{\mathrm{nuc}} +\begin{bmatrix}
 F^{AA} & 0\\
0 & 0 
\end{bmatrix} \begin{bmatrix}
\gamma^{A} & 0\\
0 & \gamma^{B} 
\end{bmatrix} + \begin{bmatrix}
0 & 0\\
0 & J^{BB}  
\end{bmatrix} \begin{bmatrix}
\gamma^{A} & 0\\
0 & \gamma^{B} 
\end{bmatrix} + \begin{bmatrix}
E_{xc}^{AA} & E_{xc}^{AB}\\
E_{xc}^{BA} & E_{xc}^{BB} 
\end{bmatrix} \begin{bmatrix}
\gamma^{A} & 0\\
0 & \gamma^{B} 
\end{bmatrix}- \begin{bmatrix}
E_{xc}^{AA} & 0\\
0 & 0 
\end{bmatrix} \begin{bmatrix}
\gamma^{A} & 0\\
0 & \gamma^{B} 
\end{bmatrix}$$ 

thus

$$-\frac{\nabla^{2}}{2} + v_{\mathrm{I}}^{\mathrm{nuc}} + v_{\mathrm{II}}^{\mathrm{nuc}} + \bigg( \begin{bmatrix}
 F^{AA} & 0\\
0 & 0 
\end{bmatrix} + \begin{bmatrix}
0 & 0\\
0 & J^{BB}  
\end{bmatrix} + \begin{bmatrix}
E_{xc}^{AA} & E_{xc}^{AB}\\
E_{xc}^{BA} & E_{xc}^{BB} 
\end{bmatrix} - \begin{bmatrix}
E_{xc}^{AA} & 0\\
0 & 0 
\end{bmatrix} \bigg)\begin{bmatrix}
\gamma^{A} & 0\\
0 & \gamma^{B} 
\end{bmatrix}$$ 



In [5]:
Fock = F0 + F1
        # transofrm Fock matrix to orthonormal basis
        Fockprime = np.dot(np.conjugate(S_half).T,np.dot(Fock, S_half))

        # diagonalise Fock matrix
        evalFockprime, Cprime = np.linalg.eig(Fockprime)

        idx = evalFockprime.argsort()
        evalFockprime = evalFockprime[idx]
        Cprime = Cprime[:,idx]

        ## construct new SCF eigenvector matrix
        C_new = S_half@Cprime 

        ## form new density matrix
        Dnew=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao), dtype=complex) 
        N=molA.nelectron + molB.nelectron
        for j in range(molA.nao+molB.nao):
            for k in range(molA.nao+molB.nao):
                for a in range(int(N/2)):
                    Dnew[j,k] += C_new[j,a]*C_new[k,a]

                    
        D_mat_list.append(Dnew)
        error = sp.linalg.norm(D_mat_list[-1] - D_mat_list[-2]) / sp.linalg.norm(D_mat_list[-2])
        D_mat_rho1 = Dnew    
        

IndentationError: unexpected indent (<ipython-input-5-ab4b20781e91>, line 3)

In [ ]:
def frozen_calc(gamma_T, F_full, S_full, active_sys, nao_A):
    
    
#     𝛾Total= [ 𝛾_act 0]
#             [ 0 𝛾_emb]
#   
    # isolated RKS on active system
    if (active_sys!='A') or (active_sys!='B'):
        raise ValueError(f"unknown system {active_sys}")
        
    
    gamma_A = gamma_T[:nao_A, :nao_A]
    Fock_A = F_full[:nao_A, :nao_A]
    Sovlp_A =S_full[:nao_A, :nao_A]
    
    gamma_B = gamma_T[nao_A:, nao_A:]
    Fock_B = F_full[nao_A:, nao_A:]
    
    
    if (active_sys=='A'):
        
        F_isolated = np.zeros(F_full.shape)
        F_isolated[:nao_A, :nao_A]=Fock_A
        
    else:
        pass
    
        
        # isolated RKS on active system
        Fockprime = np.dot(np.conjugate(S_half).T,np.dot(Fock, S_half))

        # diagonalise Fock matrix
        evalFockprime, Cprime = np.linalg.eig(Fock_A)
        
        eig_vals, eig_vecs = LA.eig(Sovlp_A)
        U_dag =np.conjugate(eig_vecs).T
        U=eig_vecs
        L=U_dag @ Sovlp @ U
        # # Form S^U(-0.5)
        S_half = U @ LA.fractional_matrix_power(L, -0.5) @ U_dag

        
                         
    
    
    
    
    # active_and_embed_KE = full system kinetic energy (mol_AB.intor_symmetric('int1e_kin')) 
    #order important! active defined first
    # 
    
    # isolated active system
    active_nuclear_energy = active_RKS_obj.energy_nuc()
    active_coloumb, active_exchange = active_RKS_obj.get_jk(dm=active_rho_dm)
    
    # embedded systems
    emded_nuclear_energy = embed_RKS_obj.energy_nuc()
    emded_coloumb = embed_RKS_obj.get_j(dm=embed_rho_dm)
    
    ### non additive parts!
    
    ## exhange
    # E_XC = E_xc(rho_active + rho_embed) - E_xc(rho_active) - E_xc(rho_embed)
    rho_full = np.block([
                        [active_rho_dm,        np.zeros((active_rho_dm.shape[0], embed_rho_dm.shape[1]))],
                        [np.zeros((embed_rho_dm.shape[0], active_rho_dm.shape[1])),        embed_rho_dm]
                        ])
    full_XC = full_System_RKS_obj.get_k(dm=rho_full)
    act_XC= active_RKS_obj.get_k(dm=active_rho_dm)
    emb_XC= embed_RKS_obj.get_k(dm=embed_rho_dm)
    non_add_exchange = full_XC-act_XC-emb_XC
    
    # select part of matrix that works with lin alg
    relevent_non_add_xc = non_add_exchange[:active_rho_dm.shape[0],:active_rho_dm.shape[1]]
    
    ## kinetic
    non_add_kinetic = active_and_embed_KE-active_KE-embed_KE
     
    
    

In [ ]:
from scipy import linalg as LA

def DFT_in_DFT(molA, molB, max_iter=50, tol=1e-5):
    
    mol_AB = molA + molB
    
    #### individual DFT systems
    # System A
    mol_A_KSDFT = scf.RKS(molA)
    mol_A_KSDFT.xc = 'lda,vwn'
    
    # System B
    mol_B_KSDFT = scf.RKS(molB)
    mol_B_KSDFT.xc = 'lda,vwn'
    
    # combined systems
    mol_T_KSDFT = scf.RKS(mol_AB)
    mol_T_KSDFT.xc = 'lda,vwn'
    
    S_full =mol_AB.intor_symmetric('int1e_ovlp')
    F_full = mol_AB.intor('int2e')

    
    i = 0
    error=tol*10
    converged_A=False
    converged_B=False
    while ((converged_A is False) and (converged_B is False)) or (i < max_iter):
        
        if i==0:
            rho1= mol_A_KSDFT.init_guess_by_atom()
            rho2= np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
        
        

In [ ]:
from scipy import linalg as LA

def DFT_in_DFT(molA, molB, max_iter=50, tol=1e-5):
    
    mol_AB = molA + molB
    
    
#     nuclear_V = mol_AB.intor_symmetric('int1e_nuc')
    kinetic_T = mol_AB.intor_symmetric('int1e_kin')
    
    mol_T_KSDFT = scf.RKS(mol_AB)
    mol_T_KSDFT.xc = 'lda,vwn'
    Sovlp = mol_T_KSDFT.get_ovlp()
    
    ## orthogonalizing matrix S^-0.5
    # U†SU = L
    eig_vals, eig_vecs = LA.eig(Sovlp)
    U_dag =np.conjugate(eig_vecs).T
    U=eig_vecs
    L=U_dag @ Sovlp @ U
    # # Form S^U(-0.5)
    S_half = U @ LA.fractional_matrix_power(L, -0.5) @ U_dag
    
    # individual DFT calcs
    mol_A_KSDFT = scf.RKS(molA)
    mol_A_KSDFT.xc = 'lda,vwn'
    mol_A_KSDFT.small_rho_cutoff = 1e-20
    mol_A_KSDFT.kernel()
    gamma_A = mol_A_KSDFT.make_rdm1()
    kinetic_A_T = molA.intor_symmetric('int1e_kin')
    
    mol_B_KSDFT = scf.RKS(molB)
    mol_B_KSDFT.xc = 'lda,vwn'
    mol_B_KSDFT.small_rho_cutoff = 1e-20
    mol_B_KSDFT.kernel()
    gamma_B = mol_B_KSDFT.make_rdm1()
    kinetic_B_T = molB.intor_symmetric('int1e_kin')
      
    i = 0
    error=tol*10
    D_mat_list=[]
    while (error > tol) and (i < max_iter):
        
        # emtpy density matrix
        
        
        
        if i==0:
            D_mat_rho1=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
            D_mat_rho2=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
            
            # rho1=gamma_sysA
            # rho2=0
            D_mat_rho1[:molA.nao, :molA.nao] = gamma_A

            D_mat_list.append(D_mat_rho1+D_mat_rho2)
        else:
            if i%2==0: #system A == rho1 (changing) and B FIXED
                D_mat_rho2=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
                D_mat_rho2[molA.nao:, molA.nao:] = D_mat_rho1.copy()[molA.nao:, molA.nao:] # old dmat B
                
                D_mat_rho1=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
                D_mat_rho1[:molA.nao, :molA.nao]=gamma_A # system A

                
            else: #system B == rho1 (changing) and A FIXED
                D_mat_rho2=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
                D_mat_rho2[:molA.nao, :molA.nao] = D_mat_rho1.copy()[:molA.nao, :molA.nao] # old dmat A
   
                D_mat_rho1=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
                D_mat_rho1[molA.nao:, molA.nao:]=gamma_B #system B
                
        
        
        F0 = mol_T_KSDFT.get_hcore() + mol_T_KSDFT.get_veff(dm=D_mat_rho1)
        F1 = mol_T_KSDFT.get_j(dm=D_mat_rho2) + mol_T_KSDFT.get_k(dm=D_mat_rho1+D_mat_rho2)\
             - mol_T_KSDFT.get_k(dm=D_mat_rho2) 
        
        
        Fock = F0 + F1
        # transofrm Fock matrix to orthonormal basis
        Fockprime = np.dot(np.conjugate(S_half).T,np.dot(Fock, S_half))

        # diagonalise Fock matrix
        evalFockprime, Cprime = np.linalg.eig(Fockprime)

        idx = evalFockprime.argsort()
        evalFockprime = evalFockprime[idx]
        Cprime = Cprime[:,idx]

        ## construct new SCF eigenvector matrix
        C_new = S_half@Cprime 

        ## form new density matrix
        Dnew=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao), dtype=complex) 
        N=molA.nelectron + molB.nelectron
        for j in range(molA.nao+molB.nao):
            for k in range(molA.nao+molB.nao):
                for a in range(int(N/2)):
                    Dnew[j,k] += C_new[j,a]*C_new[k,a]

                    
        D_mat_list.append(Dnew)
        error = sp.linalg.norm(D_mat_list[-1] - D_mat_list[-2]) / sp.linalg.norm(D_mat_list[-2])
        D_mat_rho1 = Dnew    
        
#         Dold= D_mat_rho1+D_mat_rho2
#         error = sp.linalg.norm(Dnew - Dold) / sp.linalg.norm(Dold)
#         D_mat_rho1 = Dnew
        
        print(i, error)
#         print('###')
#         print(Dnew)
#         print('###')
#         print('')
        i += 1
        
    
    if i == max_iter:
        print("SCF Did Not Converge")
    else:
        print("SCF Converged")

    return Dnew, error
    

In [ ]:
nuclear_VAB = gto.intor_cross('int1e_nuc', mol_A, mol_B)
kinetic_TAB = gto.intor_cross('int1e_kin', mol_A, mol_B)


# AO overlap between two molecules
overlap_SAB = gto.intor_cross('int1e_ovlp', mol_A, mol_B)



In [ ]:
# # 2-electron integrals over different molecules. E.g. a,c of (ab|cd) on one molecule
# # and b on another molecule and d on the third molecule.
# #
# mol1 = mol
# mol2 = gto.M(atom='He', basis='ccpvdz')
# mol3 = gto.M(atom='O', basis='sto-3g')

# mol123 = mol1 + mol2 + mol3
# eri = mol123.intor('int2e', shls_slice=(0, mol1.nbas,
#                                         mol1.nbas, mol1.nbas+mol2.nbas,
#                                         0, mol1.nbas,
#                                         mol1.nbas+mol2.nbas, mol123.nbas))

In [ ]:
mol_AB = mol_A + mol_B
# shls_slice : 8-element list
#(ish_start, ish_end, jsh_start, jsh_end, ksh_start, 
# ksh_end, lsh_start, lsh_end)

# a,c (ab|cd) on one molecule A and bd on molecule B


eri_aaaa = mol_AB.intor('int2e', shls_slice=(0, mol_A.nbas,
                                       0, mol_A.nbas,
                                       0, mol_A.nbas,
                                       0, mol_A.nbas))

# (xy|wz) : x,y, on molecule A and w,z on molecule B 
eri_aaab = mol_AB.intor('int2e', shls_slice=(0, mol_A.nbas,
                                        0, mol_A.nbas,
                                        0, mol_A.nbas,
                                        mol_A.nbas, mol_A.nbas+mol_B.nbas))

# (xy|wz) : x,y, on molecule B and w,z on molecule A 
eri_aabb = mol_AB.intor('int2e', shls_slice=(0, mol_A.nbas,
                                        0, mol_A.nbas,
                                        mol_A.nbas, mol_A.nbas+mol_B.nbas,
                                         mol_A.nbas, mol_A.nbas+mol_B.nbas))

# (xy|wz) : x,w, on molecule B and y,z on molecule A
eri_abab = mol_AB.intor('int2e', shls_slice=(0, mol_A.nbas,
                                        mol_A.nbas, mol_A.nbas+mol_B.nbas,
                                       0, mol_A.nbas,
                                         mol_A.nbas, mol_A.nbas+mol_B.nbas))


# (xy|wz) : x,y, on molecule B and w,z on molecule A 
eri_abbb = mol_AB.intor('int2e', shls_slice=(0, mol_A.nbas,
                                        mol_A.nbas, mol_A.nbas+mol_B.nbas,
                                        mol_A.nbas, mol_A.nbas+mol_B.nbas,
                                         mol_A.nbas, mol_A.nbas+mol_B.nbas))

# (xy|wz) : x,y, w, z  on molecule B 
eri_bbbb = mol_AB.intor('int2e', shls_slice=(mol_A.nbas, mol_A.nbas+mol_B.nbas,
                                       mol_A.nbas, mol_A.nbas+mol_B.nbas,
                                       mol_A.nbas, mol_A.nbas+mol_B.nbas,
                                       mol_A.nbas, mol_A.nbas+mol_B.nbas))

testAA = mol_A.intor('int2e')
print(np.allclose(eri_aaaa, testAA))

testBB = mol_B.intor('int2e')
print(np.allclose(eri_bbbb, testBB))


In [ ]:
# applying all symmetry!

eri_total = np.zeros((mol_AB.nao_nr(), 
                      mol_AB.nao_nr(), 
                      mol_AB.nao_nr(), 
                      mol_AB.nao_nr()))

nao_A =  mol_A.nao_nr()# number atomic orbs system A
nao_B =  mol_B.nao_nr()# number atomic orbs system B

# sys A
eri_total[:nao_A,:nao_A,:nao_A,:nao_A]= eri_aaaa

# single B
eri_total[:nao_A,:nao_A,:nao_A,nao_A:]= eri_aaab #ijkl=aaab
eri_total[:nao_A,:nao_A,nao_A:,:nao_A]= np.einsum('ijkl->ijlk', eri_aaab)#ijlk=aaba
eri_total[:nao_A,nao_A:,:nao_A,:nao_A]= np.einsum('ijkl->klij', eri_aaab)#iljk=abaa
eri_total[nao_A:, :nao_A,:nao_A,:nao_A]= np.einsum('ijkl->lkji', eri_aaab)#lijk=baaa

# double B sym
eri_total[:nao_A,:nao_A,nao_A:,nao_A:]= eri_aabb#ijkl=aabb
eri_total[nao_A:,nao_A:,:nao_A,:nao_A]= np.einsum('ijkl->klij', eri_aabb)#iljk=bbaa

# double B anti-sym
eri_total[:nao_A,nao_A:,:nao_A,nao_A:]= eri_abab#ijkl=abab
eri_total[:nao_A,nao_A:,nao_A:,:nao_A]= np.einsum('ijkl->ijlk', eri_abab)#ilkj=abba
eri_total[nao_A:, :nao_A,:nao_A,nao_A:]= np.einsum('ijkl->jikl', eri_abab)#lijk=baab
eri_total[nao_A:, :nao_A,nao_A:,:nao_A]= np.einsum('ijkl->jilk', eri_abab)#lijk=baba


# triple B
eri_total[:nao_A,nao_A:,nao_A:,nao_A:]= eri_abbb #ijkl=abbb
eri_total[nao_A:,:nao_A,nao_A:,nao_A:]= np.einsum('ijkl->jikl', eri_abbb)#ijlk=babb
eri_total[nao_A:,nao_A:,:nao_A,nao_A:]= np.einsum('ijkl->klij', eri_abbb)#iljk=bbab
eri_total[nao_A:,nao_A:,nao_A:,:nao_A]= np.einsum('ijkl->klji', eri_abbb)#lijk=bbba

# sys B
eri_total[nao_A:,nao_A:,nao_A:,nao_A:]= eri_bbbb

TEST = mol_AB.intor('int2e')
np.allclose(eri_total, TEST)

# Monomolcular DFT-in-DFT freeze thaw

- iterative in monomolcular basis using freeze thaw methed
    - Chemical Physics Letters 248 (1996) 71-76
    
note that:

[J. Chem. Theory Comput. 2018, 14, 1928−1942 Article]


$$\gamma^{\text{Total}}=\begin{bmatrix}
\gamma^{A} & 0\\
0 & \gamma^{B} 
\end{bmatrix}$$

$$F=\begin{bmatrix}
F^{AA} & F^{AB}\\
F^{BA} & F^{BB} 
\end{bmatrix}$$

$$S=\begin{bmatrix}
S^{AA} & S^{AB}\\
S^{BA} & S^{BB} 
\end{bmatrix}$$

In [ ]:
D_mat = np.zeros((mol_A.nao+mol_B.nao, mol_A.nao+mol_B.nao))

D_mat.shape

In [ ]:
2%2

In [ ]:
from scipy import linalg as LA

def DFT_in_DFT(molA, molB, max_iter=50, tol=1e-5):
    
    mol_AB = molA + molB
    
    
#     nuclear_V = mol_AB.intor_symmetric('int1e_nuc')
    kinetic_T = mol_AB.intor_symmetric('int1e_kin')
    
    mol_T_KSDFT = scf.RKS(mol_AB)
    mol_T_KSDFT.xc = 'lda,vwn'
    Sovlp = mol_T_KSDFT.get_ovlp()
    
    ## orthogonalizing matrix S^-0.5
    # U†SU = L
    eig_vals, eig_vecs = LA.eig(Sovlp)
    U_dag =np.conjugate(eig_vecs).T
    U=eig_vecs
    L=U_dag @ Sovlp @ U
    # # Form S^U(-0.5)
    S_half = U @ LA.fractional_matrix_power(L, -0.5) @ U_dag
    
    # individual DFT calcs
    mol_A_KSDFT = scf.RKS(molA)
    mol_A_KSDFT.xc = 'lda,vwn'
    mol_A_KSDFT.small_rho_cutoff = 1e-20
    mol_A_KSDFT.kernel()
    gamma_A = mol_A_KSDFT.make_rdm1()
    kinetic_A_T = molA.intor_symmetric('int1e_kin')
    
    mol_B_KSDFT = scf.RKS(molB)
    mol_B_KSDFT.xc = 'lda,vwn'
    mol_B_KSDFT.small_rho_cutoff = 1e-20
    mol_B_KSDFT.kernel()
    gamma_B = mol_B_KSDFT.make_rdm1()
    kinetic_B_T = molB.intor_symmetric('int1e_kin')
      
    i = 0
    error=tol*10
    D_mat_list=[]
    while (error > tol) and (i < max_iter):
        
        # emtpy density matrix
        
        
        
        if i==0:
            D_mat_rho1=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
            D_mat_rho2=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
            
            # rho1=gamma_sysA
            # rho2=0
            D_mat_rho1[:molA.nao, :molA.nao] = gamma_A

            D_mat_list.append(D_mat_rho1+D_mat_rho2)
        else:
            if i%2==0: #system A == rho1 (changing) and B FIXED
                D_mat_rho2=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
                D_mat_rho2[molA.nao:, molA.nao:] = D_mat_rho1.copy()[molA.nao:, molA.nao:] # old dmat B
                
                D_mat_rho1=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
                D_mat_rho1[:molA.nao, :molA.nao]=gamma_A # system A

                
            else: #system B == rho1 (changing) and A FIXED
                D_mat_rho2=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
                D_mat_rho2[:molA.nao, :molA.nao] = D_mat_rho1.copy()[:molA.nao, :molA.nao] # old dmat A
   
                D_mat_rho1=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
                D_mat_rho1[molA.nao:, molA.nao:]=gamma_B #system B
                
        
        
        F0 = mol_T_KSDFT.get_hcore() + mol_T_KSDFT.get_veff(dm=D_mat_rho1)
        F1 = mol_T_KSDFT.get_j(dm=D_mat_rho2) + mol_T_KSDFT.get_k(dm=D_mat_rho1+D_mat_rho2)\
             - mol_T_KSDFT.get_k(dm=D_mat_rho2) 
        
        
        Fock = F0 + F1
        # transofrm Fock matrix to orthonormal basis
        Fockprime = np.dot(np.conjugate(S_half).T,np.dot(Fock, S_half))

        # diagonalise Fock matrix
        evalFockprime, Cprime = np.linalg.eig(Fockprime)

        idx = evalFockprime.argsort()
        evalFockprime = evalFockprime[idx]
        Cprime = Cprime[:,idx]

        ## construct new SCF eigenvector matrix
        C_new = S_half@Cprime 

        ## form new density matrix
        Dnew=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao), dtype=complex) 
        N=molA.nelectron + molB.nelectron
        for j in range(molA.nao+molB.nao):
            for k in range(molA.nao+molB.nao):
                for a in range(int(N/2)):
                    Dnew[j,k] += C_new[j,a]*C_new[k,a]

                    
        D_mat_list.append(Dnew)
        error = sp.linalg.norm(D_mat_list[-1] - D_mat_list[-2]) / sp.linalg.norm(D_mat_list[-2])
        D_mat_rho1 = Dnew    
        
#         Dold= D_mat_rho1+D_mat_rho2
#         error = sp.linalg.norm(Dnew - Dold) / sp.linalg.norm(Dold)
#         D_mat_rho1 = Dnew
        
        print(i, error)
#         print('###')
#         print(Dnew)
#         print('###')
#         print('')
        i += 1
        
    
    if i == max_iter:
        print("SCF Did Not Converge")
    else:
        print("SCF Converged")

    return Dnew, error
    

In [ ]:
dmat, err = DFT_in_DFT(mol_A, mol_B, max_iter=50)

# mol_T = mol_A + mol_B
# mol_T_KSDFT = scf.RKS(mol_T)
# mol_T_KSDFT.xc = 'lda,vwn'

# F = mol_T_KSDFT.get_hcore() + mol_T_KSDFT.get_veff(dm=dmat)

# ## orthogonalizing matrix S^-0.5
# # U†SU = L
# eig_vals, eig_vecs = LA.eig(Sovlp)
# U_dag =np.conjugate(eig_vecs).T
# U=eig_vecs
# L=U_dag @ Sovlp @ U
# # # Form S^U(-0.5)
# S_half = U @ LA.fractional_matrix_power(L, -0.5) @ U_dag


In [ ]:
def DFT_A_active_B_envirn(active_rho, environ_rho):
    
    F0 = mol_T_KSDFT.get_hcore() + mol_T_KSDFT.get_veff(dm=active_rho)
    F1 = mol_T_KSDFT.get_j(dm=environ_rho) + mol_T_KSDFT.get_k(dm=active_rho+environ_rho)\
             - mol_T_KSDFT.get_k(dm=environ_rho) + kinetic_T
    

In [ ]:
l

In [ ]:
CORRECT

In [ ]:
S_Ascf_modified

In [ ]:
S_Ascf_modified.get_hcore = lambda *args: hcore_modified;
S_Ascf_modified.run();

In [ ]:
# from scipy import linalg as LA

# def DFT_in_DFT(molA, molB, max_iter=50, tol=1e-5):
    
#     mol_AB = molA + molB
    
#     mol_T_KSDFT = scf.RKS(mol_AB)
#     mol_T_KSDFT.xc = 'lda,vwn'
#     Sovlp = mol_T_KSDFT.get_ovlp()
    
#     ## orthogonalizing matrix S^-0.5
#     # U†SU = L
#     eig_vals, eig_vecs = LA.eig(Sovlp)
#     U_dag =np.conjugate(eig_vecs).T
#     U=eig_vecs
#     L=U_dag @ Sovlp @ U
#     # # Form S^U(-0.5)
#     S_half = U @ LA.fractional_matrix_power(L, -0.5) @ U_dag
    
#     # individual DFT calcs
#     mol_A_KSDFT = scf.RKS(molA)
#     mol_A_KSDFT.xc = 'lda,vwn'
#     mol_A_KSDFT.small_rho_cutoff = 1e-20
#     mol_A_KSDFT.kernel()
#     gamma_A = mol_A_KSDFT.make_rdm1()
    
#     mol_B_KSDFT = scf.RKS(molB)
#     mol_B_KSDFT.xc = 'lda,vwn'
#     mol_B_KSDFT.small_rho_cutoff = 1e-20
#     mol_B_KSDFT.kernel()
#     gamma_B = mol_B_KSDFT.make_rdm1()
    
      
#     i = 0
#     error=tol*10
#     while (error > tol) and (i < max_iter):
        
#         # emtpy density matrix
#         D_mat_rho1= np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
#         D_mat_rho2= np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
        
#         if i==0:
#             rho1 = mol_A_KSDFT.init_guess_by_atom()
#             rho2 = np.zeros((molB.nao, molB.nao))

#         else:
#             if i%2==0: #system A
#                 rho2 = rho1.copy() # old dmat
#                 rho1 = gamma_A # system A
                
#             else: #system B
#                 rho2 = rho1.copy()  # old dmat
#                 rho1 =  gamma_B #system B
        
#         D_mat_rho1[:molA.nao, :molA.nao] = rho1
#         D_mat_rho2[molA.nao:, molA.nao:] = rho2
        
#         F0 = mol_T_KSDFT.get_hcore() + mol_T_KSDFT.get_veff(dm=D_mat_rho1)
#         F1 = mol_T_KSDFT.get_j(dm=D_mat_rho2) + mol_T_KSDFT.get_k(dm=D_mat_rho1+D_mat_rho2) - mol_T_KSDFT.get_k(dm=D_mat_rho2)
        
        
#         Fock = F0 + F1
#         # transofrm Fock matrix to orthonormal basis
#         Fockprime = np.dot(np.conjugate(S_half).T,np.dot(Fock, S_half))

#         # diagonalise Fock matrix
#         evalFockprime, Cprime = np.linalg.eig(Fockprime)

#         idx = evalFockprime.argsort()
#         evalFockprime = evalFockprime[idx]
#         Cprime = Cprime[:,idx]

#         ## construct new SCF eigenvector matrix
#         C_new = S_half@Cprime 

#         ## form new density matrix
#         Dnew=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao), dtype=complex) 
#         N=molA.nelectron + molB.nelectron
#         for j in range(molA.nao+molB.nao):
#             for k in range(molA.nao+molB.nao):
#                 for a in range(int(N/2)):
#                     Dnew[j,k] += C_new[j,a]*C_new[k,a]

#         Dold= D_mat_rho1+D_mat_rho2
#         error = sp.linalg.norm(Dnew - Dold) / sp.linalg.norm(Dold)
        
#         D_mat_rho1 = Dnew
        
#         print(i, error)
#         print('###')
#         print(Dnew)
#         print('###')
#         print('')
#         i += 1
        
    
#     if i == max_iter:
#         print("SCF Did Not Converge")
#     else:
#         print("SCF Converged")

#     return Dnew, error
    

In [ ]:
Sovlpmol_T_KSDFT.get_ovlp()

In [ ]:
mol_T_KSDFT.get_k(dm=mol_T_KSDFT.make_rdm1())

In [ ]:
mol_B_KSDFT = scf.RKS(mol_B)
mol_B_KSDFT.xc = 'lda,vwn'
mol_B_KSDFT.small_rho_cutoff = 1e-20
mol_B_KSDFT.kernel()
gamma_B = mol_B_KSDFT.make_rdm1()

In [ ]:
dir(mol_T_KSDFT)

In [ ]:
mol_T_KSDFT.xc

In [ ]:
mol_B_KSDFT.get_veff(dm=gamma_B)

In [ ]:
sum(mol_B_KSDFT.mo_coeff)/2

In [ ]:
dir(mol_A)

In [ ]:
from scipy import linalg as LA

def DFT_in_DFT(molA, molB, max_iter=50, tol=1e-5):
    
    mol_AB = molA + molB
    
    
#     nuclear_V = mol_AB.intor_symmetric('int1e_nuc')
    kinetic_T = mol_AB.intor_symmetric('int1e_kin')
    
    mol_T_KSDFT = scf.RKS(mol_AB)
    mol_T_KSDFT.xc = 'lda,vwn'
    Sovlp = mol_T_KSDFT.get_ovlp()
    
    ## orthogonalizing matrix S^-0.5
    # U†SU = L
    eig_vals, eig_vecs = LA.eig(Sovlp)
    U_dag =np.conjugate(eig_vecs).T
    U=eig_vecs
    L=U_dag @ Sovlp @ U
    # # Form S^U(-0.5)
    S_half = U @ LA.fractional_matrix_power(L, -0.5) @ U_dag
    
    # individual DFT calcs
    mol_A_KSDFT = scf.RKS(molA)
    mol_A_KSDFT.xc = 'lda,vwn'
    mol_A_KSDFT.small_rho_cutoff = 1e-20
    mol_A_KSDFT.kernel()
    gamma_A = mol_A_KSDFT.make_rdm1()
    kinetic_A_T = molA.intor_symmetric('int1e_kin')
    
    mol_B_KSDFT = scf.RKS(molB)
    mol_B_KSDFT.xc = 'lda,vwn'
    mol_B_KSDFT.small_rho_cutoff = 1e-20
    mol_B_KSDFT.kernel()
    gamma_B = mol_B_KSDFT.make_rdm1()
    kinetic_B_T = molB.intor_symmetric('int1e_kin')
      
    i = 0
    error=tol*10
    D_mat_list=[]
    while (error > tol) and (i < max_iter):
        
        # emtpy density matrix
        
        
        
        if i==0:
            D_mat_rho1=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
            D_mat_rho2=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
            
            # rho1=gamma_sysA
            # rho2=0
            D_mat_rho1[:molA.nao, :molA.nao] = gamma_A

            D_mat_list.append(D_mat_rho1+D_mat_rho2)
        else:
            if i%2==0: #system a == rho1 (changing) and B FIXED
                
                D_mat_rho2 = D_mat_rho1.copy()# old dmat
                
                D_mat_rho1=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
                D_mat_rho1[:molA.nao, :molA.nao]=gamma_A # system A

                
            else: #system B == rho1 (changing) and A FIXED
                D_mat_rho2 = D_mat_rho1.copy()# old dmat
   
                D_mat_rho1=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
                D_mat_rho1[molA.nao:, molA.nao:]=gamma_B #system B
                
        active=
        environ=
        
        F0 = mol_T_KSDFT.get_hcore() + mol_T_KSDFT.get_veff(dm=D_mat_rho1)
        F1 = mol_T_KSDFT.get_j(dm=D_mat_rho2) + mol_T_KSDFT.get_k(dm=D_mat_rho1+D_mat_rho2)\
             - mol_T_KSDFT.get_k(dm=D_mat_rho2) + kinetic_T
        
        
        Fock = F0 + F1
        # transofrm Fock matrix to orthonormal basis
        Fockprime = np.dot(np.conjugate(S_half).T,np.dot(Fock, S_half))

        # diagonalise Fock matrix
        evalFockprime, Cprime = np.linalg.eig(Fockprime)

        idx = evalFockprime.argsort()
        evalFockprime = evalFockprime[idx]
        Cprime = Cprime[:,idx]

        ## construct new SCF eigenvector matrix
        C_new = S_half@Cprime 

        ## form new density matrix
        Dnew=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao), dtype=complex) 
        N=molA.nelectron + molB.nelectron
        for j in range(molA.nao+molB.nao):
            for k in range(molA.nao+molB.nao):
                for a in range(int(N/2)):
                    Dnew[j,k] += C_new[j,a]*C_new[k,a]

                    
        D_mat_list.append(Dnew)
        error = sp.linalg.norm(D_mat_list[-1] - D_mat_list[-2]) / sp.linalg.norm(D_mat_list[-2])
        D_mat_rho1 = Dnew    
        print(i, error)
        i += 1
        
    
    if i == max_iter:
        print("SCF Did Not Converge")
    else:
        print("SCF Converged")

    return Dnew, error

DFT_in_DFT(mol_A, mol_B, max_iter=50)

In [ ]:
S_Ascf_modified.get_hcore = lambda *args: hcore_modified;
S_Ascf_modified.run();


ks.make_rdm1()

In [ ]:
mol_AB = mol_A + mol_B


#     nuclear_V = mol_AB.intor_symmetric('int1e_nuc')
kinetic_T = mol_AB.intor_symmetric('int1e_kin')

mol_T_KSDFT = scf.RKS(mol_AB)
mol_T_KSDFT.xc = 'lda,vwn'
mol_T_KSDFT.make_rdm1()
mol_T_KSDFT.make_rdm1()

$$J_{\mu\nu} = \sum_{\lambda\sigma} \left(\mu\nu|\lambda\sigma\right) \left(D_{\lambda\sigma}\right)$$

$$K_{\mu\nu} = \sum_{\lambda\sigma} \left(\mu\lambda|\nu\sigma\right) D_{\lambda\sigma}$$

$$D_{\mu\nu} = \sum_{i\in occ} C_{\mu i} C_{i\nu}^{\dagger}$$

In [ ]:
mol_Total_KSDFT = scf.RKS(mol_AB)
mol_Total_KSDFT.xc = 'lda,vwn'
mol_Total_KSDFT.small_rho_cutoff = 1e-20
mol_Total_KSDFT.kernel()
# gamma_T = mol_Total_KSDFT.make_rdm1()
mol_Total_KSDFT

In [ ]:
# full matrices
J,K = scf.hf.dot_eri_dm(TEST, 
                  mol_Total_KSDFT.make_rdm1(),
                  hermi=0, 
                  with_j=True, 
                  with_k=True)

J_manual,K_manual = scf.hf.dot_eri_dm(eri_total,  # <- note here
                  mol_Total_KSDFT.make_rdm1(),
                  hermi=0, 
                  with_j=True, 
                  with_k=True)


np.allclose(J, J_manual)

In [ ]:
Fock_manual = 0.5*(J_manual - K_manual) + mol_Total_KSDFT.get_hcore()
F = 0.5*(mol_Total_KSDFT.get_j() - mol_Total_KSDFT.get_k()) + mol_Total_KSDFT.get_hcore() 

np.allclose(F, Fock_manual)

In [ ]:
x = mol_Total_KSDFT.get_veff()
y= mol_Total_KSDFT.get_j(mol_Total_KSDFT.make_rdm1()) - mol_Total_KSDFT.get_k(mol_Total_KSDFT.make_rdm1())*.5

np.allclose(x, y)
y

In [ ]:
# subsystem matrices
J,K = scf.hf.dot_eri_dm(TEST, 
                  mol_Total_KSDFT.make_rdm1(),
                  hermi=0, 
                  with_j=True, 
                  with_k=True)

J_manual,K_manual = scf.hf.dot_eri_dm(eri_total,  # <- note here
                  mol_Total_KSDFT.make_rdm1(),
                  hermi=0, 
                  with_j=True, 
                  with_k=True)


np.allclose(J, J_manual)

In [ ]:
F_total = mol_Total_KSDFT.get_hcore() + J_manual + K_manual

In [ ]:
mol_B_KSDFT = scf.RKS(mol_B)
mol_B_KSDFT.xc = 'lda,vwn'
mol_B_KSDFT.small_rho_cutoff = 1e-20
mol_B_KSDFT.kernel()
gamma_B = mol_B_KSDFT.make_rdm1()

In [ ]:
# J. Chem. Theory Comput. 2017, 13, 1503−1508, DOI: 10.1021/acs.jctc.7b00034 
# equation 7

# S_BA
overlap_SBA = gto.intor_cross('int1e_ovlp', mol_B,mol_A)
# F_AB 𝛾_B S_B
F_ab_𝛾b_S_ba = np.dot(F_total[:nao_A, nao_A:], gamma_B.dot(overlap_SBA))

# P_b = -0.5(F_AB 𝛾_B S_BA + S_AB 𝛾_B F_BA)
P_B = -0.5*(F_ab_𝛾b_S_ba + F_ab_𝛾b_S_ba.T)
P_B.shape

In [ ]:
# J. Chem. Theory Comput. 2017, 13, 1503−1508, DOI: 10.1021/acs.jctc.7b00034 
# equation 6

overlap_SAA =mol_A.intor_symmetric('int1e_ovlp')
overlap_SAB = gto.intor_cross('int1e_ovlp', mol_A, mol_B)
overlap_SBA = gto.intor_cross('int1e_ovlp', mol_B,mol_A)
overlap_SBB=mol_B.intor_symmetric('int1e_ovlp')

S_full = np.block([
    [overlap_SAA,  overlap_SAB],
    [overlap_SBA, overlap_SBB]])

# F_AB 𝛾_B S_B
gamma_T = mol_Total_KSDFT.make_rdm1()
F_ab_𝛾T_S_ba = np.dot(F_total, gamma_T.dot(S_full))

F_ab_𝛾T_S_ba.shape

In [ ]:
basis_A = mol_Total_KSDFT.mo_coeff[:,:nao_A]
basis_B = mol_Total_KSDFT.mo_coeff[:,nao_A:]

# equation (3) J. Chem. Phys. 139, 024103 (2013)
summer=np.zeros((basis_B.shape[0],basis_B.shape[0]))
for column_i in range(basis_B.shape[1]):
    summer+=np.outer(basis_B[:,column_i], basis_B[:,column_i])

PB=np.dot(mol_Total_KSDFT.mo_coeff.conj().T,
          summer.dot(mol_Total_KSDFT.mo_coeff))

In [ ]:
P_aa_B=np.dot(basis_A.conj().T,
          summer.dot(basis_A))

P_ab_B=np.dot(basis_A.conj().T,
          summer.dot(basis_B))

P_ba_B=np.dot(basis_B.conj().T,
          summer.dot(basis_A))

P_bb_B=np.dot(basis_B.conj().T,
          summer.dot(basis_B))


P_B = np.block([
    [P_aa_B,  P_ab_B],
    [P_ba_B, P_bb_B]])

np.allclose(PB, P_B)

In [ ]:
gamma_T = mol_Total_KSDFT.make_rdm1()
# PB.dot(gamma_T)
# PB.dot(mol_Total_KSDFT.mo_coeff)

In [ ]:
h_core=mol_Total_KSDFT.get_hcore()
# v_eff = mol_Total_KSDFT.get_veff() 
# v_eff = mol_Total_KSDFT.get_veff() +F_ab_𝛾T_S_ba

In [ ]:
# mol_Total_KSDFT.get_hcore()

In [ ]:
HF_Sa_embedded = scf.RHF(mol_AB) 
mol_Total_KSDFT.xc = 'lda,vwn'
mol_Total_KSDFT.conv_tol= 1e-6
mol_Total_KSDFT.run()
E_HF_no_embedding = mol_Total_KSDFT.e_tot

mol_Total_KSDFT.h_core = lambda *args: PB + h_core # <--- h_core_embed
mol_Total_KSDFT.kernel()

E_HF_with_embedding = mol_Total_KSDFT.e_tot

print('')
print('')
print('Energy with standard Hamiltonian = ', E_HF_no_embedding)
print('Energy with embedded Hamiltonian = ',E_HF_with_embedding)

In [ ]:
gamma_T = np.block([
    [gamma_A,  np.zeros((gamma_A.shape[0], gamma_B.shape[1]))],

    [np.zeros((gamma_B.shape[0], gamma_A.shape[1])), gamma_B]])


C = np.block([
    [mol_A_KSDFT.mo_coeff,  np.zeros((mol_A_KSDFT.mo_coeff.shape[0], mol_B_KSDFT.mo_coeff.shape[1]))],
    [np.zeros((mol_B_KSDFT.mo_coeff.shape[0], mol_A_KSDFT.mo_coeff.shape[1])), mol_B_KSDFT.mo_coeff]])

C.shape

In [ ]:
mol_Total_KSDFT = scf.RKS(mol_Total)
mol_Total_KSDFT.xc = 'lda,vwn'
mol_Total_KSDFT.small_rho_cutoff = 1e-20
mol_Total_KSDFT.kernel()
# gamma_T = mol_Total_KSDFT.make_rdm1()
mol_Total_KSDFT

In [ ]:
mol_Total = gto.Mole(atom=mol_A.atom + "; " +mol_B.atom,
               basis="sto3g",
               charge=mol_A.charge+mol_B.charge,
               spin=mol_A.spin+mol_B.spin)

mol_Total.build()

- (J. Chem. Theory Comput. 2020, 16, 2284−2295)

$$S^{AB}=<\chi^{A}|\chi^{B}>$$

In [ ]:

S_ab = gto.intor_cross('int1e_ovlp', mol_A, mol_B) # int1e_ovlp_sph
# np.around(S_ab, 1)
S_ab.shape

$$F^{AB}=<\chi^{A}|F|\chi^{B}>$$

- where $F$ is the full system Fock operator

keywords (https://sunqm.github.io/pyscf/modules/gto.html#pyscf.gto.moleintor.getints)

https://sunqm.github.io/pyscf/theory/scf.html

In [ ]:
# F = mol_Total.get_fock()

$$J_{\lambda \sigma} = \sum_{\lambda \sigma} (\mu \nu|\lambda \sigma) (P_{\lambda \sigma})$$

where $P_{\lambda \sigma} = \sum_{i \in occ}C_{\mu i} C_{i \nu}^{\dagger}$

In [ ]:
mol_A.

In [ ]:
S_AA= np.inner(mol_A_KSDFT.mo_coeff, mol_A_KSDFT.mo_coeff)
S_AB= np.inner(mol_A_KSDFT.mo_coeff, mol_A_KSDFT.mo_coeff)
S_BA= np.inner(mol_A_KSDFT.mo_coeff, mol_A_KSDFT.mo_coeff)
S_BB= np.inner(mol_A_KSDFT.mo_coeff, mol_A_KSDFT.mo_coeff)



In [ ]:
np.inner(mol_A_KSDFT.mo_coeff, mol_B_KSDFT.mo_coeff)

In [ ]:
mol_A_KSDFT.mo_coeff.shape

mol_B_KSDFT.mo_coeff.shape

In [ ]:
mol_Total_KSDFT = scf.RKS(mol_Total)
mol_Total_KSDFT.xc = 'lda,vwn'
mol_Total_KSDFT.small_rho_cutoff = 1e-20
mol_Total_KSDFT.kernel()
gamma_T = mol_Total_KSDFT.make_rdm1()

In [ ]:
mol_A_KSDFT = scf.RKS(mol_A)
mol_A_KSDFT.xc = 'lda,vwn'
mol_A_KSDFT.small_rho_cutoff = 1e-20
mol_A_KSDFT.kernel()
gamma_A = mol_A_KSDFT.make_rdm1()

In [ ]:
mol_B_KSDFT = scf.RKS(mol_B)
mol_B_KSDFT.xc = 'lda,vwn'
mol_B_KSDFT.small_rho_cutoff = 1e-20
mol_B_KSDFT.kernel()

In [ ]:
print(eri_ab.shape)
print(eri_ba.shape)

np.allclose(eri_ab[0,0,:15,:15], eri_ba[0,0,:15,:15].conj().T)

In [ ]:
eri_aa.shape

In [ ]:
eri_aa = gto.intor_cross('int2e', mol_A, mol_A) # eri

In [ ]:
mol_Total_KSDFT.get_fock().shape

In [ ]:
np.allclose(mol_Total_KSDFT.get_fock(), t[:15,:15,0,0])

In [ ]:
t = gto.intor_cross('int2e', mol_Total, mol_Total) # eri
t.shape

In [ ]:
mol_A_KSDFT.mo_coeff.shape

In [ ]:
# mol_A.intor('int2e', aosym='s8')
mol_A.intor('int2e').shape

In [ ]:
eri_aa = gto.intor_cross('int2e', mol_A, mol_A) # eri
eri_ab = gto.intor_cross('int2e', mol_A, mol_B) # eri
eri_ba = gto.intor_cross('int2e', mol_B, mol_A) # eri
eri_bb = gto.intor_cross('int2e', mol_B, mol_B) # eri


two_elec = np.block([
    [eri_aa,  eri_ab],
    [eri_ba, eri_bb]])

# two_elec = np.zeros((15,15,15,15))
# for i in range(eri.shape[0]):
#     for j in range(eri.shape[1]):
#         for k in range(eri.shape[2]):
#             for l in range(eri.shape[3]):
#                 two_elec[i,j,k,l]=eri[i,j,k,l]



# block_dm = np.block([
#     [gamma_A,  np.zeros((gamma_A.shape[0], gamma_B.shape[1]))],

#     [np.zeros((gamma_B.shape[0], gamma_A.shape[1])), gamma_B]])

# j, k = scf.hf.dot_eri_dm(two_elec, dm=block_dm)

In [ ]:
gamma_B.shape

In [ ]:
eri_ab.shape

In [ ]:
gamma_B.shape

In [ ]:
eri_ab = gto.intor_cross('int2e', mol_A, mol_B) # eri
# dm_ab=np.zeros(eri_ab.shape)
dm_ab=np.zeros((15,15))
shape_A=gamma_A.shape[0]
shape_B=gamma_B.shape[0]
two_elec = np.zeros((15,15,15,15))
for i in range(shape_A):
    for j in range(shape_A):
        for k in range(shape_B):
            for l in range(shape_B):
#                 dm_ab[i,j,k,l]=gamma_A[i,]+gamma_B[k,l]
                print(i,j,k,l)
dm_ab.shape
                
# j, k = scf.hf.dot_eri_dm(two_elec, dm=block_dm)

In [ ]:
scf.hf.get_veff(mol_A+mol_B, dm0, hermi=0)

In [ ]:

eri = gto.intor_cross('int2e', mol_A, mol_B) # eri
eri.shape

In [ ]:
eri = gto.intor_cross('int2e', mol_A, mol_B) # eri

two_elec = np.zeros((15,15,15,15))
for i in range(eri.shape[0]):
    for j in range(eri.shape[1]):
        for k in range(eri.shape[2]):
            for l in range(eri.shape[3]):
                two_elec[i,j,k,l]=eri[i,j,k,l]



block_dm = np.block([
    [gamma_A,  np.zeros((gamma_A.shape[0], gamma_B.shape[1]))],

    [np.zeros((gamma_B.shape[0], gamma_A.shape[1])), gamma_B]])

j, k = scf.hf.dot_eri_dm(two_elec, dm=block_dm)

In [ ]:
F = 

$$P^{B}= -\frac{1}{2} \big(F^{AB} \gamma^{B}S^{BA} + S^{AB} \gamma^{B}F^{BA} \big)$$

In [ ]:
gamma_B = mol_B_KSDFT.make_rdm1()

P_B = -0.5 * (np.dot(F_ab, gamma_B.dot(S_ab.T)))

In [ ]:
one_electron_ints = gto.intor_cross('int1e_nuc', mol_A, mol_B)
one_electron_ints.shape

In [ ]:
from pyscf.lo import ibo
mol = gto.M(atom='H 0 0 0; F 0 0 1', basis='unc-sto3g')
mf = scf.RHF(mol).run()
pm = ibo.PM(mol, mf.mo_coeff[:,mf.mo_occ>0])
loc_orb = pm.kernel()

In [ ]:
S_ab = gto.intor_cross('int1e_ovlp', mol_A, mol_B) # int1e_ovlp_sph
# np.around(S_ab, 1)
S_ab.shape

In [ ]:
two_e = gto.intor_cross('int2e', mol_A, mol_B) # int1e_ovlp_sph
# np.around(S_ab, 1)
two_e.shape

In [ ]:
two_e = gto.intor_cross('int2e', mol_A, mol_A) # int1e_ovlp_sph
# np.around(S_ab, 1)
two_e.shape

In [ ]:
mol_A_two_ints = mol_A.intor('int2e', aosym='s8')
mol_A_two_ints.shape

In [ ]:
mol_B_two_ints = mol_B.intor('int2e', aosym='s8')
mol_B_two_ints.shape

In [ ]:
(s22, numpy.dot(s21, mo1), strict_sym_pos=False)

In [ ]:
np.dot(s21, mol_B)

In [ ]:
# P_B = scf.project_mo_nr2nr(mol_B, moA, mol_B)

s22 = mol_B.intor_symmetric('int1e_ovlp')
s21 = gto.intor_cross('int1e_ovlp', mol_B, mol_A)

sp.linalg.solve(s22, np.dot(s21, mol_A_KSDFT.mo_coeff), sym_pos=False).shape

In [ ]:
mol_A_KSDFT.mo_coeff.shape

In [ ]:
mol_B.intor_symmetric('int2e').shape

In [ ]:
mol_B.nao
mol_B.nelec

In [ ]:
55*55

In [ ]:
gto.intor_cross('int2e', mol_B, mol_B).shape

In [ ]:
eri_ab = gto.intor_cross('int2e', mol_A, mol_B) # eri
eri_ab.shape

In [ ]:
eri_ab = gto.intor_cross('int2e', mol_A, mol_B) # eri
same_same = 

In [ ]:

S_ab = gto.intor_cross('int1e_ovlp', mol_A, mol_A) # int1e_ovlp_sph
# np.around(S_ab, 1)
S_ab.shape

test = mol_A.intor_symmetric('int1e_ovlp')

test==S_ab

In [ ]:
A = mol_A.intor_symmetric('int2e')
B = mol_B.intor_symmetric('int2e')
A.shape

In [ ]:
from scipy import linalg as LA

def DFT_in_DFT(molA, molB, max_iter=50, tol=1e-5):
    
    mol_AB = molA + molB
    
    
#     nuclear_V = mol_AB.intor_symmetric('int1e_nuc')
    kinetic_T = mol_AB.intor_symmetric('int1e_kin')
    
    mol_T_KSDFT = scf.RKS(mol_AB)
    mol_T_KSDFT.xc = 'lda,vwn'
    Sovlp = mol_T_KSDFT.get_ovlp()
    
    ## orthogonalizing matrix S^-0.5
    # U†SU = L
    eig_vals, eig_vecs = LA.eig(Sovlp)
    U_dag =np.conjugate(eig_vecs).T
    U=eig_vecs
    L=U_dag @ Sovlp @ U
    # # Form S^U(-0.5)
    S_half = U @ LA.fractional_matrix_power(L, -0.5) @ U_dag
    
    # individual DFT calcs
    mol_A_KSDFT = scf.RKS(molA)
    mol_A_KSDFT.xc = 'lda,vwn'
    mol_A_KSDFT.small_rho_cutoff = 1e-20
    mol_A_KSDFT.kernel()
    gamma_A = mol_A_KSDFT.make_rdm1()
    kinetic_A_T = molA.intor_symmetric('int1e_kin')
    
    mol_B_KSDFT = scf.RKS(molB)
    mol_B_KSDFT.xc = 'lda,vwn'
    mol_B_KSDFT.small_rho_cutoff = 1e-20
    mol_B_KSDFT.kernel()
    gamma_B = mol_B_KSDFT.make_rdm1()
    kinetic_B_T = molB.intor_symmetric('int1e_kin')
      
    i = 0
    error=tol*10
    D_mat_list=[]
    D_mat_A=[]
    D_mat_B=[]
    while (error > tol) and (i < max_iter):
        
        # emtpy density matrix
        
        
        
        if i==0:
            D_mat_rho1=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
            D_mat_rho2=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
            
            # rho1=gamma_sysA
            # rho2=0
            D_mat_rho1[:molA.nao, :molA.nao] = gamma_A

            D_mat_list.append(D_mat_rho1+D_mat_rho2)
        else:
            if i%2==0: #system A == rho1 (changing) and B FIXED
                D_mat_rho2=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
                D_mat_rho2[molA.nao:, molA.nao:] = D_mat_rho1.copy()[molA.nao:, molA.nao:] # old dmat B
                
                D_mat_rho1=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
                D_mat_rho1[:molA.nao, :molA.nao]=gamma_A # system A

                
            else: #system B == rho1 (changing) and A FIXED
                D_mat_rho2=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
                D_mat_rho2[:molA.nao, :molA.nao] = D_mat_rho1.copy()[:molA.nao, :molA.nao] # old dmat A
   
                D_mat_rho1=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
                D_mat_rho1[molA.nao:, molA.nao:]=gamma_B #system B
                
        
        
        F0 = mol_T_KSDFT.get_hcore() + mol_T_KSDFT.get_veff(dm=D_mat_rho1) - kinetic_T
        F1 = mol_T_KSDFT.get_j(dm=D_mat_rho2) + mol_T_KSDFT.get_k(dm=D_mat_rho1+D_mat_rho2)\
             - mol_T_KSDFT.get_k(dm=D_mat_rho2) 
        
        
        Fock = F0 + F1
        
        if i%2==0:
            # system A
            Fock_embed = F0[:molA.nao, molA.nao:] + F1[molA.nao:, molA.nao:]
            S_half_embed = S_half[molA.nao:, molA.nao:]
        else:
            # system B
            Fock_embed = F0[molA.nao:, molA.nao:] + F1[molA.nao:, molA.nao:]
            S_half_embed = S_half[molA.nao:, molA.nao:]
            
            
        
        # transofrm Fock matrix to orthonormal basis
        Fockprime = np.dot(np.conjugate(S_half_embed).T,np.dot(Fock_embed, S_half_embed))

        # diagonalise Fock matrix
        evalFockprime, Cprime = np.linalg.eig(Fockprime)

        idx = evalFockprime.argsort()
        evalFockprime = evalFockprime[idx]
        Cprime = Cprime[:,idx]

        ## construct new SCF eigenvector matrix
        C_new = S_half_embed@Cprime 

        ## form new density matrix
        
        
        if i%2==0:
            # system A
            Dnew_sub=np.zeros((molA.nao, molA.nao), dtype=complex) 
            N_A=molA.nelectron
            for j in range(molA.nao):
                for k in range(molA.nao):
                    for a in range(int(N_A/2)):
                        Dnew_sub[j,k] += C_new[j,a]*C_new[k,a]
        else:
            # system B
            Dnew_sub=np.zeros((molB.nao, molB.nao), dtype=complex) 
            N_B = molB.nelectron
            for j in range(molB.nao):
                for k in range(molB.nao):
                    for a in range(int(N_B/2)):
                        Dnew_sub[j,k] += C_new[j,a]*C_new[k,a]  
        
        Dnew = np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
        if i%2==0:
            Dnew[:molA.nao, :molA.nao]=Dnew_sub
        else:
            Dnew[molA.nao:, molA.nao:]=Dnew_sub
        Dnew+= D_mat_rho2
        
        D_mat_list.append(Dnew)
        error = sp.linalg.norm(D_mat_list[-1] - D_mat_list[-2]) / sp.linalg.norm(D_mat_list[-2])
        D_mat_rho1 = Dnew    
        
#         Dold= D_mat_rho1+D_mat_rho2
#         error = sp.linalg.norm(Dnew - Dold) / sp.linalg.norm(Dold)
#         D_mat_rho1 = Dnew
        
        print(i, error)
#         print('###')
#         print(Dnew)
#         print('###')
#         print('')
        i += 1
        
    
    if i == max_iter:
        print("SCF Did Not Converge")
    else:
        print("SCF Converged")

    return Dnew, error

x,y=DFT_in_DFT(mol_A, mol_B, max_iter=50, tol=1e-10)

In [ ]:
KSDFT.energy_elec(dm=x)
x

In [ ]:
KSDFT.energy_elec(dm=gamma_T)

In [ ]:
KSDFT = scf.RKS(mol_A+mol_B)
KSDFT.xc = 'lda,vwn'
KSDFT.small_rho_cutoff = 1e-20
KSDFT.kernel()
gamma_T = KSDFT.make_rdm1()
KSDFT.get_veff()

In [ ]:
from scipy import linalg as LA

def DFT_in_DFT(molA, molB, max_iter=50, tol=1e-5):
    
    mol_AB = molA + molB
    
    
#     nuclear_V = mol_AB.intor_symmetric('int1e_nuc')
    kinetic_T = mol_AB.intor_symmetric('int1e_kin')
    
    mol_T_KSDFT = scf.RKS(mol_AB)
    mol_T_KSDFT.xc = 'lda,vwn'
    Sovlp = mol_T_KSDFT.get_ovlp()
    
    ## orthogonalizing matrix S^-0.5
    # U†SU = L
    eig_vals, eig_vecs = LA.eig(Sovlp)
    U_dag =np.conjugate(eig_vecs).T
    U=eig_vecs
    L=U_dag @ Sovlp @ U
    # # Form S^U(-0.5)
    S_half = U @ LA.fractional_matrix_power(L, -0.5) @ U_dag
    
    # individual DFT calcs
    mol_A_KSDFT = scf.RKS(molA)
    mol_A_KSDFT.xc = 'lda,vwn'
    mol_A_KSDFT.small_rho_cutoff = 1e-20
    mol_A_KSDFT.kernel()
    gamma_A = mol_A_KSDFT.make_rdm1()
    kinetic_A_T = molA.intor_symmetric('int1e_kin')
    
    mol_B_KSDFT = scf.RKS(molB)
    mol_B_KSDFT.xc = 'lda,vwn'
    mol_B_KSDFT.small_rho_cutoff = 1e-20
    mol_B_KSDFT.kernel()
    gamma_B = mol_B_KSDFT.make_rdm1()
    kinetic_B_T = molB.intor_symmetric('int1e_kin')
      
    i = 0
    error=tol*10
    D_mat_list=[]
    while (error > tol) and (i < max_iter):
        
        # emtpy density matrix

        if i==0:
            rho1= gamma_A
            rho2= np.zeros((molB.nao, molB.nao))
        else:
            if i%2==0: #system A == rho1 (changing) and B FIXED
                rho2=D_mat_list[-1]
                rho1=gamma_A
            else: #system B == rho1 (changing) and A FIXED
                rho2=D_mat_list[-1]
                rho1=gamma_B
           
        D_mat_rho1=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
        D_mat_rho2=np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
        if i%2==0:
            D_mat_rho1[:molA.nao, :molA.nao]=rho1
            D_mat_rho2[molA.nao:, molA.nao:]=rho2
        else:
            D_mat_rho1[molA.nao:, molA.nao:]=rho1
            D_mat_rho2[:molA.nao, :molA.nao]=rho2
        
        
        F0 = mol_T_KSDFT.get_hcore() + mol_T_KSDFT.get_veff(dm=D_mat_rho1) - kinetic_T
        F1 = mol_T_KSDFT.get_j(dm=D_mat_rho2) + mol_T_KSDFT.get_k(dm=D_mat_rho1+D_mat_rho2)\
             - mol_T_KSDFT.get_k(dm=D_mat_rho2) 
        
        
        Fock = F0 + F1
        
        if i%2==0:
            # system A
            Fock_embed = F0[:, :molA.nao] + F1[:, :molA.nao]
            S_half_embed = S_half[:molA.nao, :]
            xx= np.dot(Fock_embed, S_half_embed)
            S_half_conj=S_half[:molA.nao, :]
            print(S_half_embed.conj().T.shape)
            yy = S_half_embed.conj().T.dot(xx)
        else:
            # system B
            Fock_embed = F0[:, molA.nao:] + F1[:, molA.nao:]
            S_half_embed = S_half[molA.nao:, :]
            
            
        
        # transofrm Fock matrix to orthonormal basis
        Fockprime = np.dot(np.conjugate(S_half_embed).T,np.dot(Fock_embed, S_half_embed))

        # diagonalise Fock matrix
        evalFockprime, Cprime = np.linalg.eig(Fockprime)

        idx = evalFockprime.argsort()
        evalFockprime = evalFockprime[idx]
        Cprime = Cprime[:,idx]

        ## construct new SCF eigenvector matrix
        C_new = S_half_embed@Cprime 

        ## form new density matrix
        
        
        if i%2==0:
            # system A
            Dnew_sub=np.zeros((molA.nao, molA.nao), dtype=complex) 
            N_A=molA.nelectron
            for j in range(molA.nao):
                for k in range(molA.nao):
                    for a in range(int(N_A/2)):
                        Dnew_sub[j,k] += C_new[j,a]*C_new[k,a]
        else:
            # system B
            Dnew_sub=np.zeros((molB.nao, molB.nao), dtype=complex) 
            N_B = molB.nelectron
            for j in range(molB.nao):
                for k in range(molB.nao):
                    for a in range(int(N_B/2)):
                        Dnew_sub[j,k] += C_new[j,a]*C_new[k,a]  
        
        D_mat_list.append(Dnew_sub)
        
        Dnew = np.zeros((molA.nao+molB.nao, molA.nao+molB.nao))
        if i%2==0:
            Dnew[:molA.nao, :molA.nao]=Dnew_sub
        else:
            Dnew[molA.nao:, molA.nao:]=Dnew_sub
        Dnew+= D_mat_rho2
#         E=KSDFT.energy_elec(dm=Dnew)
#         print(E)
        
        
        if i>0:
            error = sp.linalg.norm(D_mat_list[-1] - D_mat_list[-2]) / sp.linalg.norm(D_mat_list[-2])
            D_mat_rho1 = Dnew    
            print(i, error)
            
#         Dold= D_mat_rho1+D_mat_rho2
#         error = sp.linalg.norm(Dnew - Dold) / sp.linalg.norm(Dold)
#         D_mat_rho1 = Dnew
        
        
#         print('###')
#         print(Dnew)
#         print('###')
#         print('')
        i += 1
        
    
    if i == max_iter:
        print("SCF Did Not Converge")
    else:
        print("SCF Converged")

    return Dnew, error

x,y=DFT_in_DFT(mol_A, mol_B, max_iter=50, tol=1e-10)

In [ ]:
 kinetic_T = mol_AB.intor_symmetric('int1e_kin')

In [6]:
def V_eff_embed_KS_rhoI(molecule_obj_systemI, molecule_RKS_DFT_systemI, dmat_SystemI):
#     KS effective potential of the isolated subsystem I containing
#     the usual terms of the nuclear potential, the Coulomb potential of the electrons, and
#     the exchange–correlation potential
    V_R_systemI = molecule_obj_systemI.intor_symmetric('int1e_nuc')
    Veff = molecule_RKS_DFT_systemI.get_veff(dm=dmat_SystemI)
    
    return V_R_systemI + Veff

In [7]:
def V_eff_embed_KS_rhoI_rho2(molecule_obj_systemI
                             molecule_obj_systemII,
                             molecule_RKS_DFT_systemI,
                             molecule_RKS_DFT_systemII,
                             molecule_RKS_DFT_FULL,
                             dmat_SystemI,
                             dmat_SystemII):
    # the interaction of the subsystem
    # I with the frozen density and nuclei of subsystem II
    
    V_R_systemII = molecule_obj_systemII.intor_symmetric('int1e_nuc')
    J_II, K_II = molecule_RKS_DFT_systemII.get_jk(dm=dmat_SystemII)
    
    K_I = molecule_RKS_DFT_systemI.get_k(dm=dmat_SystemI)
    
    dmat_full=np.zeros((molecule_obj_systemI.nao+molecule_obj_systemII.nao,
                        molecule_obj_systemI.nao+molecule_obj_systemII.nao))
    K_full = molecule_RKS_DFT_FULL.get_veff(dm=dmat_full)
    E_XC_nadd = K_full-K_I-K_II
    
    return V_R_systemII + Veff

SyntaxError: invalid syntax (<ipython-input-7-71513dfc5ac3>, line 2)

In [ ]:
# Enter your code here. Read input from STDIN. Print output to STDOUT
from scipy.special import comb as combinations

boys = 1.09
girls = 1
# boys, girls = list(map(int, input().strip().split(' ')))
prob_boy = boys/(boys+girls)
n=6
cumlat_prob = sum(combinations(n, x) * prob_boy**x *(1-prob_boy)**(n-x) for x in range(3,7,1) )
print(round(cumlat_prob, 3))

In [ ]:
# Enter your code here. Read input from STDIN. Print output to STDOUT

# prob_reject, N_pistons = list(map(int, input().strip().split(' ')))

percent_reject, N_pistons=12,10
prob_reject=percent_reject/100
def recursive_fact(n):
    return 1 if n == 0 else n*recursive_fact(n-1)

def combinations(n, x):
    return recursive_fact(n) / (recursive_fact(x) * recursive_fact(n-x))

def binomial(N_success, T_total_N, prob_success):
    return combinations(T_total_N, N_success) * prob_success**N_success * (1-prob_success)**(T_total_N-N_success)

max_rejects=2
prob_no_more_than_2_rejects = sum(binomial(x, N_pistons, prob_reject) for x in range(1,max_rejects+1,1) )
print(round(prob_no_more_than_2_rejects, 3))
print(round(1-prob_no_more_than_2_rejects, 3))


In [ ]:
p, n = 12,10
print(round(sum([binomial(i, n, p/100) for i in range(3)]), 3))
print(round(sum([binomial(i, n, p/100) for i in range(2, n+1)]), 3))